In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from zipfile import ZipFile
import pandas as pd

with ZipFile("/content/drive/MyDrive/processed data/past five years/future_2019_2023.csv.zip", "r") as unzipped_file:
  with unzipped_file.open("future_2019_2023.csv") as csv_file:
    future_2019_2023 = pd.read_csv(csv_file)

In [ ]:
future_2019_2023.head()

,date,rp_entity_id,comnam,headline,future_ret
0,2019-01-02,24CB56,KEYCORP NEW,Total System Services Price Target Cut to $90....,0.033723
1,2019-01-02,24CB56,KEYCORP NEW,MW Worldpay stock price target cut to $90 from...,0.033723
2,2019-01-02,24CB56,KEYCORP NEW,i3 Verticals Is Maintained at Overweight by Ke...,0.033723
3,2019-01-02,24CB56,KEYCORP NEW,MW Concho Resources started at overweight with...,0.033723
4,2019-01-02,24CB56,KEYCORP NEW,Global Payments Price Target Cut to $130.00/Sh...,0.033723


In [ ]:
future_2019_2023.shape

(7378418, 5)

In [5]:
%%capture
!pip install bertopic
!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64

In [6]:
import os
from sentence_transformers import SentenceTransformer
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(future_2019_2023.headline.tolist(),show_progress_bar=True)
#save embeddings
import numpy as np
np.save('/content/drive/MyDrive/processed data/future_2019_2023_embeddings.npy', embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/230576 [00:00<?, ?it/s]

In [7]:
# load embeddings
import numpy as np
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = np.load('/content/drive/MyDrive/processed data/future_2019_2023_embeddings.npy')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
headline_list = future_2019_2023.headline.tolist()
headline_list

['Total System Services Price Target Cut to $90.00/Share From $110.00 by KeyBanc',
 'MW Worldpay stock price target cut to $90 from $120 at KeyBanc Capital',
 'i3 Verticals Is Maintained at Overweight by KeyBanc',
 'MW Concho Resources started at overweight with $118 stock price target at KeyBanc Capital',
 'Global Payments Price Target Cut to $130.00/Share From $135.00 by KeyBanc',
 'MW First Data stock price target cut to $20 from $25 at KeyBanc Capital',
 'MW I3 Verticals stock price target raised to $26 from $22 at KeyBanc Capital',
 'First Data Price Target Cut to $20.00/Share From $25.00 by KeyBanc',
 'Worldpay Is Maintained at Overweight by KeyBanc',
 'Jernigan Capital Upsizes Credit Facility to $235 Million, Extends Maturity and Adds Banks',
 'MW Total System Services stock price target cut to $90 from $110 at KeyBanc Capital',
 'MW Global Payments stock price target cut to $130 from $135 at KeyBanc Capital',
 'Worldpay Price Target Cut to $90.00/Share From $120.00 by KeyBanc',

In [ ]:
# create vocabulary in advance to release memory
import collections
from tqdm import tqdm

# Initialize a counter to keep track of word frequencies
vocab = collections.Counter()

# Create a tokenizer using CountVectorizer from sklearn
tokenizer = CountVectorizer().build_tokenizer()

# Update the vocabulary with tokens from each headline
for headline in tqdm(headline_list):
    vocab.update(tokenizer(headline))

# Filter the vocabulary to include only words that appear more than 20 times
vocab = [word for word, count in vocab.items() if count > 20]

# Get the number of unique words in the filtered vocabulary
len(vocab)

100%|██████████| 1685713/1685713 [00:11<00:00, 142431.98it/s]


40059

In [ ]:
vocab

['Asia',
 'Pacific',
 'Corporate',
 'Calendar',
 'Month',
 'Ahead',
 'CBOT',
 'Grain',
 'Warehouse',
 'Receipt',
 'Certificates',
 'Jan',
 'North',
 'American',
 'Morning',
 'Briefing',
 'Bullish',
 'Start',
 'to',
 '2023',
 'as',
 'Stock',
 'Futures',
 'Jump',
 'Here',
 'Are',
 'Barron',
 '12',
 'Best',
 'Income',
 'Investments',
 'for',
 'the',
 'New',
 'Year',
 'Barrons',
 'com',
 'Chevron',
 'Is',
 'Maintained',
 'at',
 'Overweight',
 'by',
 'Barclays',
 'Press',
 'Release',
 'Alex',
 'Wu',
 'former',
 'Managing',
 'Director',
 'of',
 'Bain',
 'Capital',
 'takes',
 'President',
 'and',
 'CFO',
 'NaaS',
 'Real',
 'Time',
 'Economics',
 'Recession',
 'Jobs',
 'China',
 'Reeling',
 'Economy',
 'The',
 'Ledger',
 'Companies',
 'Spend',
 'Record',
 'Amounts',
 'on',
 'Dividends',
 'Despite',
 'Looming',
 'Downturn',
 'Wheat',
 'Soybean',
 'Corn',
 'Prices',
 'Expected',
 'Have',
 'Choppy',
 'Commodities',
 'Roundup',
 'Berkshire',
 'Hathaway',
 'Had',
 'Strong',
 '2022',
 'What',
 'Watc

In [9]:
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
# Cluster embeddings
hdbscan_model = HDBSCAN(min_cluster_size =1000,  metric='euclidean', cluster_selection_method='eom',\
                        gen_min_span_tree=True,prediction_data=False,min_samples = 50,verbose = True)
# Vectorize
vectorizer_model = CountVectorizer(stop_words="english", min_df=0.1, max_df = 0.9, ngram_range=(1, 2))

In [ ]:
from bertopic import BERTopic

# Create BERTopic model
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  #representation_model=representation_model,

  calculate_probabilities=False,
  low_memory = True,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(future_2019_2023.headline.tolist(), embeddings)

# Show topics
topic_model.get_topic_info()

2024-05-28 21:18:55,838 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-28 22:54:32,710 - BERTopic - Dimensionality - Completed ✓
2024-05-28 22:54:32,959 - BERTopic - Cluster - Start clustering the reduced embeddings


In [ ]:
os.mkdir("/content/drive/MyDrive/processed data/future_2019_2023_topic_model")
topic_model.save("/content/drive/MyDrive/processed data/future_2019_2023_topic_model",serialization = "safetensors", save_ctfidf = True, save_embedding_model = embedding_model)

In [ ]:
Topic_model_future_2019_2023 = BERTopic.load("/content/drive/MyDrive/processed data/future_2019_2023_topic_model",embedding_model = SentenceTransformer('all-MiniLM-L6-v2'))

In [ ]:
topic_dist, _ = Topic_model_future_2019_2023.approximate_distribution(headline_list)
future_ret_topic_dist = pd.concat([future_2019_2023.drop(columns = ["rp_entity_id","headline"]),pd.DataFrame(topic_dist)],axis = 1)
future_ret_topic_dist

100%|██████████| 1686/1686 [02:24<00:00, 11.70it/s]


,date,comnam,future_ret,0,1,2,3,4,5,6,...,41,42,43,44,45,46,47,48,49,50
0,2023-01-03,AMCOR PLC,0.000840,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2023-01-03,BORGWARNER INC,-0.012990,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,2023-01-03,CHEVRON CORP NEW,-0.022748,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,2023-01-03,CHEVRON CORP NEW,-0.022748,0.0,0.0,0.0,0.05513,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.413927
4,2023-01-03,CHEVRON CORP NEW,-0.022748,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685708,2023-12-29,UNITED AIRLINES HOLDINGS INC,-0.013862,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172766,0.000000
1685709,2023-12-29,UNITED AIRLINES HOLDINGS INC,-0.016917,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.465676,0.000000
1685710,2023-12-29,UNITED AIRLINES HOLDINGS INC,-0.013862,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.192866,0.000000
1685711,2023-12-29,UNITED AIRLINES HOLDINGS INC,-0.013862,0.0,0.0,0.0,0.00000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
grouped = future_ret_topic_dist.groupby(['date',"comnam","future_ret"])
grouped_sum = grouped.sum()

In [ ]:
grouped_sum.head()

0         1         2   \
date       comnam              ret                                       
2023-01-03 3M CO                0.021264  0.418489  0.000000  0.000000   
           A E S CORP          -0.022601  0.000000  0.110206  0.000000   
           A P A CORP          -0.065553  0.000000  0.000000  0.000000   
           A T & T INC          0.017925  0.000000  1.348731  0.394251   
           ABBOTT LABORATORIES -0.001913  0.000000  0.231780  0.000000   

                                                3         4         5   \
date       comnam              ret                                       
2023-01-03 3M CO                0.021264  0.000000  0.923652  0.000000   
           A E S CORP          -0.022601  0.000000  1.465881  0.000000   
           A P A CORP          -0.065553  0.000000  0.000000  0.000000   
           A T & T INC          0.017925  0.235264  0.266891  0.377846   
           ABBOTT LABORATORIES -0.001913  0.000000  0.000000  0.000000   

                                                6         7         8   \
date       comnam              ret                                       
2023-01-03 3M CO                0.021264  1.002335  0.159857  0.000000   
           A E S CORP          -0.022601  0.000000  0.337135  0.000000   
           A P A CORP          -0.065553  0.196057  0.289191  0.000000   
           A T & T INC          0.017925  0.472688  0.062149  0.000000   
           ABBOTT LABORATORIES -0.001913  0.039705  0.044085  0.280165   

                                                9   ...        52   53  \
date       comnam              ret                  ...                  
2023-01-03 3M CO                0.021264  2.204187  ...  1.909602  0.0   
           A E S CORP          -0.022601  0.000000  ...  0.000000  0.0   
           A P A CORP          -0.065553  0.000000  ...  0.321492  0.0   
           A T & T INC          0.017925  3.104023  ...  2.335924  0.0   
           ABBOTT LABORATORIES -0.001913  0.000000  ...  0.000000  0.0   

                                                54        55   56   57   58  \
date       comnam              ret                                            
2023-01-03 3M CO                0.021264  0.000000  0.183520  0.0  0.0  0.0   
           A E S CORP          -0.022601  0.000000  0.000000  0.0  0.0  0.0   
           A P A CORP          -0.065553  0.000000  0.000000  0.0  0.0  0.0   
           A T & T INC          0.017925  0.767201  0.063532  2.0  0.0  0.0   
           ABBOTT LABORATORIES -0.001913  0.000000  0.120236  0.0  0.0  0.0   

                                                59   60   61  
date       comnam              ret                            
2023-01-03 3M CO                0.021264  0.000000  0.0  0.0  
           A E S CORP          -0.022601  0.000000  0.0  0.0  
           A P A CORP          -0.065553  0.000000  0.0  0.0  
           A T & T INC          0.017925  1.057668  0.0  1.0  
           ABBOTT LABORATORIES -0.001913  0.000000  0.0  0.0  

[5 rows x 62 columns]

In [ ]:
topic_num = topic_dist.shape[1]
topics = np.array(grouped_sum)
ret = []
for ind in list(grouped_sum.index):
  ret.append(ind[2])
returns = np.array(ret).reshape(-1,1)
from sklearn.linear_model import LinearRegression
bert_model = LinearRegression(fit_intercept=True)
bert_model.fit(topics,returns)
bert_model.score(topics,returns)

import matplotlib.pyplot as plt
import statsmodels.api as sm

# Add a constant to the independent variable for the intercept
X = sm.add_constant(topics)
# Fit the regression model
model_sm = sm.OLS(returns, X).fit()
# Print the summary of the regression
print(model_sm.summary())
print("\n")

#plot the coefficients
plt.figure(figsize=(10, 5))
plt.bar(range(len(bert_model.coef_[0])), bert_model.coef_[0])
plt.xticks(range(len(bert_model.coef_[0])), ["topic_" + str(i + 1) for i in range(topic_num)], rotation=90)
plt.title("Coefficients of Topics")
plt.show()


0.0025850341541956423